In [2]:
from pathlib import Path
import pandas as pd


In [3]:
# Categorías a analizar
categories = pd.read_csv("../../data/categories/categories_lvl_3.csv", sep=";", header=None)
categories_list = categories.iloc[:, 0].astype(str).str.upper().tolist()

# Cogemos inferencia de llava
llava_classification = pd.read_csv("../../data/inference/lvl_3_wo_descriptions.csv", sep=";", header=0)
llava_classification["img"] = llava_classification["img"].apply(lambda x: x.split("/")[-1])
llava_classification['category_llava'] = llava_classification['category_llava'].apply(lambda x: x.upper())
llava_classification = llava_classification[["img","category_llava"]]
# Las categorías que no están en la lista de categorías, han sido malas inferencias. 

# 1. Hay varias categorías que se pueden afinar. Por ejemplo, el modelo ha puesto SPIRITUAL, y no la frase completa
# de Spiritual, symbolic and related connotations
llava_classification["category_llava"] = llava_classification["category_llava"].replace("SPIRITUAL","SPIRITUAL, SYMBOLIC AND RELATED CONNOTATIONS") 
llava_classification["category_llava"] = llava_classification["category_llava"].replace("SYMBOLIC AND RELATED CONNOTATIONS","SPIRITUAL, SYMBOLIC AND RELATED CONNOTATIONS") 

llava_classification["category_llava"] = llava_classification["category_llava"].apply(
                    lambda cat: cat if cat in categories_list or cat == "NOT VALID" or cat == "NOT RELEVANT" else "BAD_INFERENCE"
                    )

# Mergeamos con el resultado de etiquetado manual
manual_classification = pd.read_csv("../../data/inference/lvl_3_manual_annotation.csv", sep=";", header=0)
manual_classification['manual_category'] = manual_classification['manual_category'].apply(lambda x: x.upper())
# Unimos ambos por imagen
result = llava_classification.merge(manual_classification[["img","manual_category"]],on="img",how="left")


print(result.head(10).to_string())
result.to_csv("lvlm_vs_manual.csv",sep=";")



                                       img          category_llava                               manual_category
0  aiguestortes_NA_1366_47701956991__b.jpg                 SHELTER                                     LANDFORMS
1  aiguestortes_NA_1633_31399087577__b.jpg                  PLANTS                       VEGETATION AND HABITATS
2  aiguestortes_NA_1633_50177463812__b.jpg  TERRESTRIAL ACTIVITIES                        TERRESTRIAL ACTIVITIES
3  aiguestortes_NA_1690_52491627395__b.jpg                  BRIDGE                             TRACKS AND TRAILS
4  aiguestortes_NA_2008_19242320949__b.jpg                  PLANTS                                        PLANTS
5  aiguestortes_NA_4531_49900773177__b.jpg                 ANIMALS                                       ANIMALS
6  aiguestortes_NA_4792_24046167316__b.jpg    HERITAGE AND CULTURE                                  NOT RELEVANT
7    guadarrama_NA_1089_20943542884__b.jpg                  CITIES                            TO

In [4]:
nan_rows = result[result.isna().any(axis=1)]
print(nan_rows)

Empty DataFrame
Columns: [img, category_llava, manual_category]
Index: []


In [5]:
from sklearn.metrics import classification_report

# Generar el reporte de clasificación
report = classification_report(
    result['manual_category'], 
    result['category_llava'], 
    output_dict=True, 
    zero_division=0
)

# Convertir a DataFrame y transponer para que las categorías sean el índice
df_report = pd.DataFrame(report).T

# # Mostrar el DataFrame con categorías a la izquierda y métricas como columnas
# df_report = df_report.drop(columns=["support"], errors="ignore")
df_report = df_report.drop(index=["accuracy","macro avg","weighted avg","nan"], errors="ignore")
# df_report = df_report.dropna(axis=0, how="all")
print(df_report.to_string())
df_report = df_report.round(3)
df_report.to_csv("lvlm_vs_manual_report.csv", sep=";")


                                              precision    recall  f1-score  support
ACCOMMODATION                                  0.860465  0.274074  0.415730    135.0
AIR ACTIVITIES                                 0.607143  0.680000  0.641509     25.0
ANIMALS                                        0.937158  0.695740  0.798603    493.0
BAD_INFERENCE                                  0.000000  0.000000  0.000000      0.0
BRIDGE                                         0.622449  0.863208  0.723320    212.0
CITIES                                         0.291139  0.239583  0.262857     96.0
COMMERCE FACILITIES                            0.439024  0.247423  0.316484    291.0
DAM                                            0.791667  0.322034  0.457831     59.0
DOCK                                           0.000000  0.000000  0.000000      0.0
FUNGUS                                         1.000000  0.760000  0.863636     25.0
HERITAGE AND CULTURE                           0.152980  0.804938